<a href="https://colab.research.google.com/github/Donggeon2960/AutoDrive-ML/blob/main/ImageNet_%EB%AC%BC%EC%B2%B4%EC%9D%B8%EC%8B%9D(CNN%ED%95%84%EC%9A%94%EC%97%86%EC%9D%8C%2C_%EC%BD%94%EB%93%9C_%EB%B6%84%EC%84%9D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ImageNet 사전 훈련된 CNN 모델을 활용한 이미지 분류기" 이코드는 CNN이 사용할 필요가 없는 ResNet50, VGG16, EfficientNet 등은 이미 완전히 훈련된 CNN 모델들을 사용한 코드

In [ ]:
# 📦 Google Colab용 ImageNet 1000개 클래스 분류기
# 🧠 사전 훈련된 딥러닝 모델로 즉시 객체 인식!

# 필요한 라이브러리 설치 및 임포트
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
from io import BytesIO
import cv2

*   torch: PyTorch 딥러닝 프레임워크
*   torch.nn: 신경망 구성 요소들
*   torchvision.transforms: 이미지 전처리 도구
*   torchvision.models: 사전 훈련된 모델들
*   PIL.Image: 이미지 처리 라이브러리
*   numpy: 수치 계산 라이브러리
*   matplotlib.pyplot: 그래프 및 시각화
*   requests: HTTP 요청 (이미지 다운로드용)
*   json: JSON 데이터 처리
*   io.BytesIO: 바이트 스트림 처리
*   cv2: OpenCV 이미지 처리

In [ ]:
print("✅ 라이브러리 로드 완료!")
print(f"🔥 PyTorch 버전: {torch.__version__}")
print(f"💻 CUDA 사용 가능: {torch.cuda.is_available()}")

*  라이브러리 로드 완료 메시지
*   PyTorch 버전 출력
*   GPU(CUDA) 사용 가능 여부 확인

In [ ]:
class ImageNetClassifier:
    """ImageNet 사전 훈련된 모델을 사용한 1000개 클래스 분류기"""

*   이미지 분류를 위한 클래스 정의 시작

In [ ]:
def __init__(self, model_name='resnet50'):
        print(f"🧠 {model_name} 모델 로딩 중...")
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"💻 사용 디바이스: {self.device}")

        # 모델 로드
        self.model = self.load_model(model_name)
        self.model.eval()  # 평가 모드

        # ImageNet 클래스 라벨 로드
        self.class_labels = self.load_imagenet_labels()

        # 이미지 전처리 설정
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

        print(f"✅ {model_name} 모델 준비 완료!")
        print(f"📊 분류 가능한 클래스: 1000개")

🔧 초기화 메서드 (__init__)
pythondef __init__(self, model_name='resnet50'):

클래스 생성자, 기본 모델은 ResNet50

pythonprint(f"🧠 {model_name} 모델 로딩 중...")
self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"💻 사용 디바이스: {self.device}")

모델 로딩 메시지 출력
GPU 사용 가능하면 CUDA, 아니면 CPU 사용
사용할 디바이스 출력

pythonself.model = self.load_model(model_name)
self.model.eval()

지정된 모델 로드
모델을 평가 모드로 설정 (훈련 모드 아님)

pythonself.class_labels = self.load_imagenet_labels()

ImageNet 1000개 클래스 라벨 로드

pythonself.transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
])

이미지 전처리 파이프라인 정의:

Resize(256): 이미지를 256x256으로 리사이즈
CenterCrop(224): 중앙에서 224x224 크기로 자르기
ToTensor(): PIL 이미지를 텐서로 변환
Normalize(): ImageNet 평균/표준편차로 정규화



pythonprint(f"✅ {model_name} 모델 준비 완료!")
print(f"📊 분류 가능한 클래스: 1000개")

모델 준비 완료 메시지

In [ ]:
 def load_model(self, model_name):
        """사전 훈련된 모델 로드"""
        models_dict = {
            'resnet50': models.resnet50(pretrained=True),
            'resnet101': models.resnet101(pretrained=True),
            'vgg16': models.vgg16(pretrained=True),
            'vgg19': models.vgg19(pretrained=True),
            'densenet121': models.densenet121(pretrained=True),
            'efficientnet_b0': models.efficientnet_b0(pretrained=True),
            'mobilenet_v2': models.mobilenet_v2(pretrained=True),
            'alexnet': models.alexnet(pretrained=True),
            'inception_v3': models.inception_v3(pretrained=True)
        }

        if model_name not in models_dict:
            print(f"⚠️ {model_name} 모델을 찾을 수 없습니다. ResNet50을 사용합니다.")
            model_name = 'resnet50'

        model = models_dict[model_name]
        return model.to(self.device)

모델 로드 메서드
pythondef load_model(self, model_name):
    """사전 훈련된 모델 로드"""

사전 훈련된 모델을 로드하는 메서드

pythonmodels_dict = {
    'resnet50': models.resnet50(pretrained=True),
    'resnet101': models.resnet101(pretrained=True),
    'vgg16': models.vgg16(pretrained=True),
    'vgg19': models.vgg19(pretrained=True),
    'densenet121': models.densenet121(pretrained=True),
    'efficientnet_b0': models.efficientnet_b0(pretrained=True),
    'mobilenet_v2': models.mobilenet_v2(pretrained=True),
    'alexnet': models.alexnet(pretrained=True),
    'inception_v3': models.inception_v3(pretrained=True)
}

사용 가능한 사전 훈련된 모델들의 딕셔너리
pretrained=True: ImageNet으로 사전 훈련된 가중치 사용

pythonif model_name not in models_dict:
    print(f"⚠️ {model_name} 모델을 찾을 수 없습니다. ResNet50을 사용합니다.")
    model_name = 'resnet50'

존재하지 않는 모델명이면 기본값(ResNet50)으로 변경

pythonmodel = models_dict[model_name]
return model.to(self.device)

선택된 모델을 지정된 디바이스로 이동하여 반환

In [ ]:
def load_imagenet_labels(self):
        """ImageNet 1000개 클래스 라벨 로드"""
        try:
            # ImageNet 클래스 라벨 다운로드
            url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
            response = requests.get(url)
            labels = response.text.strip().split('\n')
            print(f"📋 ImageNet 라벨 로드 완료: {len(labels)}개")
            return labels
        except:
            print("⚠️ 온라인 라벨 로드 실패. 기본 라벨을 사용합니다.")
            # 일부 주요 클래스만 포함한 기본 라벨
            return [f"class_{i}" for i in range(1000)]

라벨 로드 메서드
pythondef load_imagenet_labels(self):
    """ImageNet 1000개 클래스 라벨 로드"""

ImageNet 클래스 라벨을 로드하는 메서드

pythontry:
    url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
    response = requests.get(url)
    labels = response.text.strip().split('\n')
    print(f"📋 ImageNet 라벨 로드 완료: {len(labels)}개")
    return labels
except:
    print("⚠️ 온라인 라벨 로드 실패. 기본 라벨을 사용합니다.")
    return [f"class_{i}" for i in range(1000)]

GitHub에서 ImageNet 클래스 라벨 다운로드 시도
실패하면 기본 라벨(class_0, class_1, ...) 사용

In [ ]:
def load_image(self, image_source):
        """이미지 로드 (파일 경로 또는 URL)"""
        try:
            if isinstance(image_source, str):
                if image_source.startswith('http'):
                    # URL에서 이미지 다운로드
                    response = requests.get(image_source)
                    image = Image.open(BytesIO(response.content)).convert('RGB')
                else:
                    # 로컬 파일
                    image = Image.open(image_source).convert('RGB')
            else:
                # PIL Image 객체
                image = image_source.convert('RGB')

            return image
        except Exception as e:
            print(f"❌ 이미지 로드 실패: {e}")
            return None

이미지 로드 메서드
pythondef load_image(self, image_source):
    """이미지 로드 (파일 경로 또는 URL)"""

다양한 소스에서 이미지를 로드하는 메서드

pythontry:
    if isinstance(image_source, str):
        if image_source.startswith('http'):
            response = requests.get(image_source)
            image = Image.open(BytesIO(response.content)).convert('RGB')
        else:
            image = Image.open(image_source).convert('RGB')
    else:
        image = image_source.convert('RGB')
    return image
except Exception as e:
    print(f"❌ 이미지 로드 실패: {e}")
    return None
    
*   문자열이면서 'http'로 시작하면 URL에서 다운로드
*   그냥 문자열이면 로컬 파일 경로로 처리
*   아니면 PIL 이미지 객체로 처리
*   모든 이미지를 RGB 모드로 변환



In [ ]:
 def preprocess_image(self, image):
        """이미지 전처리"""
        if image is None:
            return None

        # PIL Image → Tensor
        tensor = self.transform(image).unsqueeze(0)  # 배치 차원 추가
        return tensor.to(self.device)

이미지 전처리 메서드
pythondef preprocess_image(self, image):
    """이미지 전처리"""
    if image is None:
        return None
    
    tensor = self.transform(image).unsqueeze(0)
    return tensor.to(self.device)

*   이미지가 None이면 None 반환
*   앞서 정의한 전처리 파이프라인 적용
*   unsqueeze(0): 배치 차원 추가 (1, 3, 224, 224)
*   텐서를 지정된 디바이스로 이동



In [ ]:
def predict(self, image_source, top_k=5):
        """이미지 분류 예측"""
        print(f"🔍 이미지 분석 중...")

        # 이미지 로드
        image = self.load_image(image_source)
        if image is None:
            return None

        original_image = image.copy()

        # 전처리
        input_tensor = self.preprocess_image(image)
        if input_tensor is None:
            return None

        # 예측
        with torch.no_grad():
            outputs = self.model(input_tensor)
            probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

        # Top-K 결과 추출
        top_prob, top_indices = torch.topk(probabilities, top_k)

        results = []
        for i in range(top_k):
            class_idx = top_indices[i].item()
            prob = top_prob[i].item()
            class_name = self.class_labels[class_idx]
            results.append({
                'rank': i + 1,
                'class_index': class_idx,
                'class_name': class_name,
                'probability': prob,
                'percentage': prob * 100
            })

        return {
            'original_image': original_image,
            'predictions': results,
            'model_info': {
                'model_name': self.model.__class__.__name__,
                'device': str(self.device),
                'total_classes': len(self.class_labels)
            }
        }

예측 메서드
pythondef predict(self, image_source, top_k=5):
    """이미지 분류 예측"""
    print(f"🔍 이미지 분석 중...")

이미지 분류를 수행하는 메서드
top_k: 상위 k개 예측 결과 반환

pythonimage = self.load_image(image_source)
if image is None:
    return None

original_image = image.copy()

이미지 로드 및 원본 이미지 복사본 보존

pythoninput_tensor = self.preprocess_image(image)
if input_tensor is None:
    return None

이미지 전처리하여 텐서로 변환

pythonwith torch.no_grad():
    outputs = self.model(input_tensor)
    probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

torch.no_grad(): 그래디언트 계산 비활성화 (메모리 절약)
모델에 입력하여 출력 획득
softmax를 적용하여 확률로 변환

pythontop_prob, top_indices = torch.topk(probabilities, top_k)

상위 k개의 확률과 인덱스 추출

pythonresults = []
for i in range(top_k):
    class_idx = top_indices[i].item()
    prob = top_prob[i].item()
    class_name = self.class_labels[class_idx]
    results.append({
        'rank': i + 1,
        'class_index': class_idx,
        'class_name': class_name,
        'probability': prob,
        'percentage': prob * 100
    })

각 예측 결과를 딕셔너리로 구성
순위, 클래스 인덱스, 클래스명, 확률, 백분율 포함

pythonreturn {
    'original_image': original_image,
    'predictions': results,
    'model_info': {
        'model_name': self.model.__class__.__name__,
        'device': str(self.device),
        'total_classes': len(self.class_labels)
    }
}

원본 이미지, 예측 결과, 모델 정보를 포함한 딕셔너리 반환

In [ ]:
 def visualize_results(self, results, show_top_k=5):
        """결과 시각화"""
        if results is None:
            print("❌ 결과가 없습니다.")
            return

        # 결과 출력
        print(f"\n🎯 ImageNet 분류 결과 (Top {show_top_k}):")
        print("=" * 60)

        predictions = results['predictions'][:show_top_k]

        for pred in predictions:
            print(f"{pred['rank']}. {pred['class_name']:<30} {pred['percentage']:.2f}%")

        print("=" * 60)

        # 시각화
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # 원본 이미지
        ax1.imshow(results['original_image'])
        ax1.set_title('Original Image', fontsize=14, fontweight='bold')
        ax1.axis('off')

        # 예측 결과 막대 그래프
        class_names = [pred['class_name'][:20] for pred in predictions]  # 이름 길이 제한
        probabilities = [pred['percentage'] for pred in predictions]
        colors = plt.cm.viridis(np.linspace(0, 1, len(predictions)))

        bars = ax2.barh(range(len(predictions)), probabilities, color=colors)
        ax2.set_yticks(range(len(predictions)))
        ax2.set_yticklabels(class_names)
        ax2.set_xlabel('Confidence (%)')
        ax2.set_title(f'Top {show_top_k} Predictions', fontsize=14, fontweight='bold')
        ax2.invert_yaxis()  # 높은 확률이 위로

        # 막대에 퍼센트 표시
        for i, (bar, prob) in enumerate(zip(bars, probabilities)):
            ax2.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
                    f'{prob:.1f}%', va='center', fontweight='bold')

        # 전체 제목
        best_prediction = predictions[0]
        plt.suptitle(f'🏆 Best Prediction: {best_prediction["class_name"]} ({best_prediction["percentage"]:.1f}%)',
                    fontsize=16, fontweight='bold')

        plt.tight_layout()
        plt.show()

        # 모델 정보
        model_info = results['model_info']
        print(f"\n📊 모델 정보:")
        print(f"   🧠 모델: {model_info['model_name']}")
        print(f"   💻 디바이스: {model_info['device']}")
        print(f"   📋 총 클래스 수: {model_info['total_classes']}")

결과 시각화 메서드
pythondef visualize_results(self, results, show_top_k=5):
    """결과 시각화"""
    if results is None:
        print("❌ 결과가 없습니다.")
        return

예측 결과를 시각화하는 메서드

pythonprint(f"\n🎯 ImageNet 분류 결과 (Top {show_top_k}):")
print("=" * 60)

predictions = results['predictions'][:show_top_k]

for pred in predictions:
    print(f"{pred['rank']}. {pred['class_name']:<30} {pred['percentage']:.2f}%")

print("=" * 60)

상위 k개 예측 결과를 텍스트로 출력
순위, 클래스명, 확률 표시

pythonfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

1x2 구조의 서브플롯 생성 (이미지 + 막대그래프)

pythonax1.imshow(results['original_image'])
ax1.set_title('Original Image', fontsize=14, fontweight='bold')
ax1.axis('off')

첫 번째 서브플롯에 원본 이미지 표시
축 제거하여 깔끔하게 표시

pythonclass_names = [pred['class_name'][:20] for pred in predictions]
probabilities = [pred['percentage'] for pred in predictions]
colors = plt.cm.viridis(np.linspace(0, 1, len(predictions)))

클래스명을 20자로 제한
확률값 리스트 생성
viridis 컬러맵으로 색상 생성

pythonbars = ax2.barh(range(len(predictions)), probabilities, color=colors)
ax2.set_yticks(range(len(predictions)))
ax2.set_yticklabels(class_names)
ax2.set_xlabel('Confidence (%)')
ax2.set_title(f'Top {show_top_k} Predictions', fontsize=14, fontweight='bold')
ax2.invert_yaxis()

수평 막대그래프 생성
y축 눈금과 라벨 설정
x축 라벨 설정
y축 순서 뒤집기 (높은 확률이 위로)

pythonfor i, (bar, prob) in enumerate(zip(bars, probabilities)):
    ax2.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
            f'{prob:.1f}%', va='center', fontweight='bold')

각 막대 오른쪽에 확률 텍스트 표시

pythonbest_prediction = predictions[0]
plt.suptitle(f'🏆 Best Prediction: {best_prediction["class_name"]} ({best_prediction["percentage"]:.1f}%)',
            fontsize=16, fontweight='bold')

전체 그래프 상단에 최고 예측 결과 표시

pythonplt.tight_layout()
plt.show()

레이아웃 조정 및 그래프 표시

In [ ]:
 def compare_models(self, image_source, model_names=['resnet50', 'vgg16', 'efficientnet_b0']):
        """여러 모델 성능 비교"""
        print(f"🔄 다중 모델 비교 시작...")

        original_model = self.model
        original_name = original_model.__class__.__name__

        results_comparison = []

        for model_name in model_names:
            print(f"\n🧠 {model_name} 모델로 예측 중...")

            # 모델 변경
            self.model = self.load_model(model_name)
            self.model.eval()

            # 예측
            result = self.predict(image_source, top_k=3)
            if result:
                best_pred = result['predictions'][0]
                results_comparison.append({
                    'model_name': model_name,
                    'best_class': best_pred['class_name'],
                    'confidence': best_pred['percentage'],
                    'top3': result['predictions']
                })

        # 원래 모델 복원
        self.model = original_model

        # 비교 결과 시각화
        if results_comparison:
            self.visualize_model_comparison(results_comparison, image_source)

        return results_comparison

모델 비교 메서드
pythondef compare_models(self, image_source, model_names=['resnet50', 'vgg16', 'efficientnet_b0']):
    """여러 모델 성능 비교"""

여러 모델로 동일 이미지를 분류하여 비교

pythonoriginal_model = self.model
original_name = original_model.__class__.__name__

원래 모델 백업

pythonresults_comparison = []

for model_name in model_names:
    print(f"\n🧠 {model_name} 모델로 예측 중...")
    
    self.model = self.load_model(model_name)
    self.model.eval()
    
    result = self.predict(image_source, top_k=3)
    if result:
        best_pred = result['predictions'][0]
        results_comparison.append({
            'model_name': model_name,
            'best_class': best_pred['class_name'],
            'confidence': best_pred['percentage'],
            'top3': result['predictions']
        })

각 모델로 예측 수행
최고 예측과 상위 3개 결과 저장

pythonself.model = original_model

원래 모델 복원

In [ ]:
def visualize_model_comparison(self, results_comparison, image_source):
        """모델 비교 결과 시각화"""
        # 이미지 로드
        image = self.load_image(image_source)

        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # 원본 이미지
        axes[0, 0].imshow(image)
        axes[0, 0].set_title('Original Image', fontsize=14, fontweight='bold')
        axes[0, 0].axis('off')

        # 각 모델별 Top 1 결과
        model_names = [result['model_name'] for result in results_comparison]
        confidences = [result['confidence'] for result in results_comparison]
        predictions = [result['best_class'][:15] for result in results_comparison]  # 이름 단축

        colors = ['red', 'blue', 'green', 'orange', 'purple'][:len(model_names)]

        bars = axes[0, 1].bar(model_names, confidences, color=colors, alpha=0.7)
        axes[0, 1].set_title('Model Confidence Comparison', fontsize=14, fontweight='bold')
        axes[0, 1].set_ylabel('Confidence (%)')
        axes[0, 1].tick_params(axis='x', rotation=45)

        # 막대에 예측 클래스와 확률 표시
        for bar, pred, conf in zip(bars, predictions, confidences):
            axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                           f'{pred}\n{conf:.1f}%', ha='center', va='bottom', fontsize=10)

        # 상세 비교 테이블
        axes[1, 0].axis('off')
        table_data = []
        for result in results_comparison:
            row = [result['model_name'], result['best_class'][:20], f"{result['confidence']:.1f}%"]
            table_data.append(row)

        table = axes[1, 0].table(cellText=table_data,
                                colLabels=['Model', 'Best Prediction', 'Confidence'],
                                cellLoc='center',
                                loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.5)
        axes[1, 0].set_title('Detailed Comparison', fontsize=14, fontweight='bold')

        # 모델별 Top 3 비교
        axes[1, 1].axis('off')
        y_pos = 0.9
        for result in results_comparison:
            axes[1, 1].text(0.05, y_pos, f"🧠 {result['model_name']}:", fontweight='bold', fontsize=12)
            y_pos -= 0.08
            for i, pred in enumerate(result['top3']):
                axes[1, 1].text(0.1, y_pos, f"{i+1}. {pred['class_name'][:25]} ({pred['percentage']:.1f}%)",
                               fontsize=10)
                y_pos -= 0.06
            y_pos -= 0.02

        axes[1, 1].set_xlim(0, 1)
        axes[1, 1].set_ylim(0, 1)
        axes[1, 1].set_title('Top 3 Predictions per Model', fontsize=14, fontweight='bold')

        plt.tight_layout()
        plt.show()

모델 비교 시각화 메서드
pythondef visualize_model_comparison(self, results_comparison, image_source):
    """모델 비교 결과 시각화"""

여러 모델의 예측 결과를 4개 패널로 구성된 그래프로 시각화

pythonimage = self.load_image(image_source)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

원본 이미지 로드
2x2 구조의 서브플롯 생성 (16x12 인치 크기)

📷 패널 1: 원본 이미지 (좌상단)
pythonaxes[0, 0].imshow(image)
axes[0, 0].set_title('Original Image', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

첫 번째 패널에 원본 이미지 표시
축과 눈금 제거하여 깔끔하게 표시

📊 패널 2: 모델별 신뢰도 막대그래프 (우상단)
pythonmodel_names = [result['model_name'] for result in results_comparison]
confidences = [result['confidence'] for result in results_comparison]
predictions = [result['best_class'][:15] for result in results_comparison]  # 이름 단축

모델명, 신뢰도, 예측 클래스명 리스트 생성
클래스명을 15자로 제한하여 가독성 향상

pythoncolors = ['red', 'blue', 'green', 'orange', 'purple'][:len(model_names)]

각 모델에 고유한 색상 할당 (최대 5개 모델까지)

pythonbars = axes[0, 1].bar(model_names, confidences, color=colors, alpha=0.7)
axes[0, 1].set_title('Model Confidence Comparison', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Confidence (%)')
axes[0, 1].tick_params(axis='x', rotation=45)

모델별 신뢰도를 수직 막대그래프로 표시
투명도 0.7로 설정하여 부드러운 외관
x축 라벨을 45도 회전하여 겹침 방지

pythonfor bar, pred, conf in zip(bars, predictions, confidences):
    axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                   f'{pred}\n{conf:.1f}%', ha='center', va='bottom', fontsize=10)

각 막대 위에 예측 클래스명과 정확한 신뢰도 표시
막대 중앙에 수평 정렬, 막대 위쪽에 수직 정렬
두 줄로 표시: 클래스명 + 신뢰도 백분율

📋 패널 3: 상세 비교 테이블 (좌하단)
pythonaxes[1, 0].axis('off')
table_data = []
for result in results_comparison:
    row = [result['model_name'], result['best_class'][:20], f"{result['confidence']:.1f}%"]
    table_data.append(row)

축 표시 제거 (테이블 전용 패널)
각 모델의 결과를 테이블 행으로 구성
클래스명을 20자로 제한

pythontable = axes[1, 0].table(cellText=table_data,
                        colLabels=['Model', 'Best Prediction', 'Confidence'],
                        cellLoc='center',
                        loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.5)
axes[1, 0].set_title('Detailed Comparison', fontsize=14, fontweight='bold')

matplotlib 테이블 생성
열 제목: 모델, 최고 예측, 신뢰도
셀 내용을 중앙 정렬
폰트 크기 10으로 고정
테이블 크기를 1.2배(가로) x 1.5배(세로)로 확대

📝 패널 4: 모델별 Top 3 예측 (우하단)
pythonaxes[1, 1].axis('off')
y_pos = 0.9
for result in results_comparison:
    axes[1, 1].text(0.05, y_pos, f"🧠 {result['model_name']}:", fontweight='bold', fontsize=12)
    y_pos -= 0.08
    for i, pred in enumerate(result['top3']):
        axes[1, 1].text(0.1, y_pos, f"{i+1}. {pred['class_name'][:25]} ({pred['percentage']:.1f}%)",
                       fontsize=10)
        y_pos -= 0.06
    y_pos -= 0.02

축 표시 제거 (텍스트 전용 패널)
y 좌표를 0.9부터 시작하여 위에서 아래로 배치
각 모델명을 굵은 글씨로 표시 (이모지 포함)
모델명 아래에 Top 3 예측을 들여쓰기하여 표시
클래스명을 25자로 제한
각 모델 사이에 추가 간격(0.02) 추가

pythonaxes[1, 1].set_xlim(0, 1)
axes[1, 1].set_ylim(0, 1)
axes[1, 1].set_title('Top 3 Predictions per Model', fontsize=14, fontweight='bold')

x, y 축 범위를 0-1로 설정 (정규화된 좌표계)
패널 제목 설정

🎨 최종 레이아웃 및 표시
pythonplt.tight_layout()
plt.show()

자동 레이아웃 조정으로 패널 간 겹침 방지
최종 그래프 화면에 표시

💡 이 메서드의 특징

4-패널 레이아웃: 원본 이미지, 막대그래프, 테이블, 텍스트 리스트로 구성
다양한 시각화: 그래프, 테이블, 텍스트를 조합하여 포괄적인 정보 제공
색상 구분: 각 모델에 고유 색상 할당으로 구별 용이
텍스트 길이 제한: 클래스명 길이를 제한하여 레이아웃 깨짐 방지
정확한 배치: 좌표를 세밀하게 계산하여 텍스트를 정확히 배치
정보 계층화: 요약 정보(막대그래프)부터 상세 정보(Top 3 리스트)까지 단계별 제공

In [ ]:
def classify_uploaded_image():
    """파일 업로드 후 ImageNet 분류"""
    print("🚀 ImageNet 1000개 클래스 분류기 시작!")
    print("=" * 60)

    # 파일 업로드
    print("📁 이미지를 업로드해주세요...")
    print("🎯 1000개 클래스 중에서 가장 유사한 객체를 찾아드립니다!")

    from google.colab import files
    uploaded = files.upload()

    if not uploaded:
        print("❌ 파일이 업로드되지 않았습니다!")
        return None

    filename = list(uploaded.keys())[0]
    print(f"📷 업로드된 파일: {filename}")

    try:
        # 분류기 생성
        classifier = ImageNetClassifier(model_name='resnet50')

        # 예측
        results = classifier.predict(filename, top_k=10)

        # 결과 시각화
        classifier.visualize_results(results, show_top_k=5)

        print("\n✅ ImageNet 분류 완료!")
        return results

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

단일 이미지 분류 함수
pythondef classify_uploaded_image():
    """파일 업로드 후 ImageNet 분류"""

Google Colab에서 파일 업로드 후 분류하는 함수

pythonfrom google.colab import files
uploaded = files.upload()

Colab의 파일 업로드 UI 사용

pythonif not uploaded:
    print("❌ 파일이 업로드되지 않았습니다!")
    return None

filename = list(uploaded.keys())[0]

업로드된 파일이 없으면 종료
첫 번째 업로드된 파일명 획득

pythonclassifier = ImageNetClassifier(model_name='resnet50')
results = classifier.predict(filename, top_k=10)
classifier.visualize_results(results, show_top_k=5)

분류기 인스턴스 생성
예측 수행 (상위 10개)
결과 시각화 (상위 5개 표시)

In [ ]:
def compare_multiple_models():
    """여러 모델로 동일 이미지 비교"""
    print("🔄 다중 모델 성능 비교!")
    print("=" * 60)

    from google.colab import files
    uploaded = files.upload()

    if not uploaded:
        print("❌ 파일이 업로드되지 않았습니다!")
        return None

    filename = list(uploaded.keys())[0]
    print(f"📷 업로드된 파일: {filename}")

    try:
        # 분류기 생성
        classifier = ImageNetClassifier()

        # 여러 모델 비교
        models_to_compare = ['resnet50', 'vgg16', 'efficientnet_b0', 'mobilenet_v2']
        comparison_results = classifier.compare_models(filename, models_to_compare)

        print("\n✅ 다중 모델 비교 완료!")
        return comparison_results

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

다중 모델 비교 함수
pythondef compare_multiple_models():
    """여러 모델로 동일 이미지 비교"""

여러 모델의 성능을 비교하는 함수

pythonmodels_to_compare = ['resnet50', 'vgg16', 'efficientnet_b0', 'mobilenet_v2']
comparison_results = classifier.compare_models(filename, models_to_compare)

비교할 모델 목록 정의
모델 비교 수행

In [ ]:
def batch_classify():
    """여러 이미지 일괄 업로드 분류"""
    print("📁 여러 이미지 일괄 분류!")
    print("=" * 60)

    from google.colab import files
    uploaded = files.upload()

    if not uploaded:
        print("❌ 파일이 업로드되지 않았습니다!")
        return None

    try:
        classifier = ImageNetClassifier()
        results_list = []

        for filename in uploaded.keys():
            print(f"\n🔍 {filename} 분석 중...")
            results = classifier.predict(filename, top_k=5)

            if results:
                # 간단한 결과 출력
                best_pred = results['predictions'][0]
                print(f"🏆 {filename}: {best_pred['class_name']} ({best_pred['percentage']:.1f}%)")
                results_list.append((filename, best_pred['class_name'], best_pred['percentage']))

        # 전체 결과 요약
        print(f"\n{'='*60}")
        print("📋 일괄 분류 결과 요약")
        print(f"{'='*60}")
        for filename, pred_class, confidence in results_list:
            print(f"📷 {filename:25} → {pred_class:20} ({confidence:.1f}%)")

        return results_list

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

일괄 분류 함수
pythondef batch_classify():
    """여러 이미지 일괄 업로드 분류"""

여러 이미지를 한 번에 분류하는 함수

pythonfor filename in uploaded.keys():
    print(f"\n🔍 {filename} 분석 중...")
    results = classifier.predict(filename, top_k=5)
    
    if results:
        best_pred = results['predictions'][0]
        print(f"🏆 {filename}: {best_pred['class_name']} ({best_pred['percentage']:.1f}%)")
        results_list.append((filename, best_pred['class_name'], best_pred['percentage']))

업로드된 모든 파일에 대해 분류 수행
각 파일의 최고 예측 결과 출력 및 저장

In [ ]:
# =============================================================================
# 메인 실행 부분
# =============================================================================

print("🎯 ImageNet 1000개 클래스 분류기 실행 옵션:")
print("1. 단일 이미지 분류: classify_uploaded_image()")
print("2. 다중 모델 비교: compare_multiple_models()")
print("3. 여러 이미지 일괄 분류: batch_classify()")
print("\n💡 추천: classify_uploaded_image() 로 시작하세요!")
print("\n🧠 사용 가능한 모델: ResNet50, VGG16, EfficientNet, MobileNet 등")
print("📊 분류 가능: 동물, 차량, 음식, 도구, 자연물 등 1000개 카테고리")
print("\n⚡ 준비 완료! 업로드 전용 ImageNet 분류기입니다!")

# 사용자가 직접 함수 호출
classify_uploaded_image()

메인 실행 부분
pythonprint("🎯 ImageNet 1000개 클래스 분류기 실행 옵션:")
print("1. 단일 이미지 분류: classify_uploaded_image()")
print("2. 다중 모델 비교: compare_multiple_models()")
print("3. 여러 이미지 일괄 분류: batch_classify()")

사용 가능한 실행 옵션 안내

pythonclassify_uploaded_image()

단일 이미지 분류 함수를 기본으로 실행